<div style="background-color:#f4f8ff; padding:16px; border-left:6px solid #1f4fd8; border-radius:6px; color:#000;">

<h2 style="margin-top:0; color:#000;">Product Report</h2>

<h4 style="color:#000;">Purpose</h4>
<ul>
  <li>This report consolidates key product metrics and behaviors.</li>
</ul>

<h4 style="color:#000;">Highlights</h4>
<ol>
  <li>Gathers essential fields such as product name, category, subcategory, and cost.</li>
  <li>Segments products by revenue to identify High-Performers, Mid-Range, or Low-Performers.</li>
  <li>Aggregates product-level metrics:
    <ul>
      <li>total orders</li>
      <li>total sales</li>
      <li>total quantity sold</li>
      <li>total customers (unique)</li>
      <li>lifespan (in months)</li>
    </ul>
  </li>
  <li>Calculates valuable KPIs:
    <ul>
      <li>recency (months since last sale)</li>
      <li>average order revenue (AOR)</li>
      <li>average monthly revenue</li>
    </ul>
  </li>
</ol>

</div>


In [6]:
query = """
WITH base_query AS (
/*---------------------------------------------------------------------------
1) Base Query: Retrieves core columns from fact_sales and dim_products
---------------------------------------------------------------------------*/
    SELECT
	    f.order_number,
        f.order_date,
		f.customer_key,
        f.sales_amount,
        f.quantity,
        p.product_key,
        p.product_name,
        p.category,
        p.subcategory,
        p.cost
    FROM gold.fact_sales f
    LEFT JOIN gold.dim_products p
        ON f.product_key = p.product_key
    WHERE order_date IS NOT NULL  -- only consider valid sales dates
),

product_aggregations AS (
/*---------------------------------------------------------------------------
2) Product Aggregations: Summarizes key metrics at the product level
---------------------------------------------------------------------------*/
SELECT
    product_key,
    product_name,
    category,
    subcategory,
    cost,
    DATEDIFF(MONTH, MIN(order_date), MAX(order_date)) AS lifespan,
    MAX(order_date) AS last_sale_date,
    COUNT(DISTINCT order_number) AS total_orders,
	COUNT(DISTINCT customer_key) AS total_customers,
    SUM(sales_amount) AS total_sales,
    SUM(quantity) AS total_quantity,
	ROUND(AVG(CAST(sales_amount AS FLOAT) / NULLIF(quantity, 0)),1) AS avg_selling_price
FROM base_query

GROUP BY
    product_key,
    product_name,
    category,
    subcategory,
    cost
)

/*---------------------------------------------------------------------------
  3) Final Query: Combines all product results into one output
---------------------------------------------------------------------------*/
SELECT TOP 100
	product_key,
	product_name,
	category,
	subcategory,
	cost,
	last_sale_date,
	DATEDIFF(MONTH, last_sale_date, GETDATE()) AS recency_in_months,
	CASE 
		WHEN total_sales > 50000 THEN 'High-Performer'
		WHEN total_sales >= 10000 THEN 'Mid-Range'
		ELSE 'Low-Performer'
	END AS product_segment,
	lifespan,
	total_orders,
	total_sales,
	total_quantity,
	total_customers,
	avg_selling_price,
/*  Average Order Revenue (AOR) */
 	CASE 
		WHEN total_orders = 0 THEN 0
		ELSE total_sales / total_orders
	END AS avg_order_revenue,

/*  Average Monthly Revenue */
	CASE
		WHEN lifespan = 0 THEN total_sales
		ELSE total_sales / lifespan
	END AS avg_monthly_revenue

FROM product_aggregations 

"""

df = pd.read_sql(query, engine)
display(HTML(df.to_html(index=False)))

product_key,product_name,category,subcategory,cost,last_sale_date,recency_in_months,product_segment,lifespan,total_orders,total_sales,total_quantity,total_customers,avg_selling_price,avg_order_revenue,avg_monthly_revenue
3,Mountain-100 Black- 38,Bikes,Mountain Bikes,1898,2011-12-27,168,High-Performer,11,49,165375,49,49,3375.0,3375,15034
4,Mountain-100 Black- 42,Bikes,Mountain Bikes,1898,2011-12-27,168,High-Performer,11,45,151875,45,45,3375.0,3375,13806
5,Mountain-100 Black- 44,Bikes,Mountain Bikes,1898,2011-12-21,168,High-Performer,11,60,202500,60,60,3375.0,3375,18409
6,Mountain-100 Black- 48,Bikes,Mountain Bikes,1898,2011-12-26,168,High-Performer,12,57,192375,57,57,3375.0,3375,16031
7,Mountain-100 Silver- 38,Bikes,Mountain Bikes,1912,2011-12-22,168,High-Performer,12,58,197200,58,58,3400.0,3400,16433
8,Mountain-100 Silver- 42,Bikes,Mountain Bikes,1912,2011-12-28,168,High-Performer,11,42,142800,42,42,3400.0,3400,12981
9,Mountain-100 Silver- 44,Bikes,Mountain Bikes,1912,2011-12-12,168,High-Performer,12,49,166600,49,49,3400.0,3400,13883
10,Mountain-100 Silver- 48,Bikes,Mountain Bikes,1912,2011-12-23,168,High-Performer,11,36,122400,36,36,3400.0,3400,11127
16,Road-150 Red- 44,Bikes,Road Bikes,2171,2011-12-28,168,High-Performer,12,281,1005418,281,281,3578.0,3578,83784
17,Road-150 Red- 48,Bikes,Road Bikes,2171,2011-12-28,168,High-Performer,12,337,1205786,337,337,3578.0,3578,100482
